<p><a name="yelp"></a></p>
### Scrape Yelp Reviews

#### Step 1: Find the pattern of url

In [85]:
from bs4 import BeautifulSoup
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
            }

response = requests.get('https://www.yelp.com/biz/shake-shack-washington-3', headers=headers).text
soup = BeautifulSoup(response, 'html.parser')

In [86]:
import re

num_reviews = soup.find('span', attrs={'class': 'review-count rating-qualifier'}).string
num_reviews = int(re.search('\d+', num_reviews).group())
print(num_reviews)

82


In [87]:
url_list = []
for i in range(0, num_reviews, 20):
    url_list.append('https://www.yelp.com/biz/shake-shack-washington-3?start='+str(i))
#print(url_list)

#### Step 2: Find all the review divs on the page

In [88]:
reviews = soup.find_all('div', attrs={'class':'review review--with-sidebar'})
print(len(reviews))

20


#### Step 3: Apply to all the pages

In [97]:
import csv
import time
import random

def scrape_single_page(reviews, csvwriter):
    for review in reviews:
        dic = {}
        try:
            username = review.find('a', attrs={'class': 'user-display-name js-analytics-click'}).text
        except:
            username = ""
        try:
            location = review.find('li', attrs={'class': 'user-location responsive-hidden-small'}).text.strip()
        except:
            location = ""
        try:
            friends_n = review.find('li', attrs={'class': 'friend-count responsive-small-display-inline-block'}).get_text().strip()
        except:
            friends_n = ""
        try:
            reviews_n = review.find('li', attrs={'class': 'review-count responsive-small-display-inline-block'}).get_text().strip()
        except:
            reviews_n = ""
        try:
            photos_n = review.find('li', attrs={'class': 'photo-count responsive-small-display-inline-block'}).get_text().strip()
        except:
            photos_n = ""
        try:
            elite = review.find('li', attrs={'class': 'is-elite responsive-small-display-inline-block'}).get_text().strip()
        except:
            elite = ""
        try:
            date = review.find('span', attrs={'class': 'rating-qualifier'}).text.strip()
        except:
            date = ""
        try:
            rating = review.find('img', attrs={'class': 'offscreen'}).get('alt')
            rating = float(re.search('\d+', rating).group())
        except:
            rating = ""
        try:
            check_in = review.find('li', attrs={'class': 'review-tags_item'}).get_text().strip()
        except:
            check_in = ""
        try:
            content = review.find('p').text
        except:
            content = ""
        try:
            useful_n = review.find('a', attrs={'class': 'ybtn ybtn--small useful js-analytics-click'}).get_text().strip()[-1]
        except:
            useful_n = ""
        try:
            date_reply = review.find('span', attrs={'class': 'bullet-after'}).get_text().strip()
        except:
            date_reply = ""
        dic['username'] = username
        dic['location'] = location
        dic['friends_n'] = friends_n
        dic['reviews_n'] = reviews_n
        dic['photos_n'] = photos_n
        dic['elite'] = elite
        dic['date'] = date
        dic['rating'] = rating
        dic['check_in'] = check_in
        dic['content'] = content
        dic['useful_n'] = useful_n
        dic['date_reply'] = date_reply
        csvwriter.writerow(dic.values())
    
with open('ss_navy_yard.csv', 'w') as csvfile:
    review_writer = csv.writer(csvfile)
    for index, url in enumerate(url_list):
        response = requests.get(url, headers=headers).text
        soup = BeautifulSoup(response, 'html.parser')
        reviews = soup.find_all('div', attrs={'class':'review review--with-sidebar'})
        scrape_single_page(reviews, review_writer)
        # Random sleep to avoid getting banned from the server
        time.sleep(random.randint(1,3))
        # Log the progress
        print('Finished page ' + str(index + 1))

Finished page 1
Finished page 2
Finished page 3
Finished page 4
Finished page 5
